## KNN Model

In [10]:
import pandas as pd
import numpy as np
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn import cross_validation
from sklearn import covariance
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor as KNN
import string
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import math
import string
import os
import datetime

In [11]:
train_data = pd.read_csv("train_imputed.csv")
train_data.head()

/Users/marinaadario/marinaadario/anaconda3/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tax_value,property_size,zestimate_amount,bathrooms,tax_year,latitude,bedrooms,year_built,home_size,longitude,...,precinct_string_26.0,precinct_string_28.0,precinct_string_30.0,precinct_string_32.0,precinct_string_33.0,precinct_string_34.0,precinct_string_5.0,precinct_string_6.0,precinct_string_7.0,precinct_string_9.0
0,1674000,2250,7332356,1.0,2015,40.724385,2,1900,10800,-73.980120,...,0,0,0,0,0,0,0,0,0,1
1,2290000,2000,1991289,1.0,2015,40.724332,4,1910,3374,-73.980007,...,0,0,0,0,0,0,0,0,0,1
2,2837000,650,1225236,1.0,2015,40.724225,1,1876,650,-73.979720,...,0,0,0,0,0,0,0,0,0,1
3,6701000,2250,12785150,1.0,2015,40.724125,2,1901,4168,-73.979495,...,0,0,0,0,0,0,0,0,0,1
4,1670000,2250,7615912,1.0,2015,40.724095,1,1950,8720,-73.979414,...,0,0,0,0,0,0,0,0,0,1


In [12]:
x_train = train_data.drop(['zestimate_amount'],1)
y_train = train_data['zestimate_amount']

In [13]:
def k_fold_knn(x, y, n_neighbors, num_folds):
    
    kfold = cross_validation.KFold(len(x), num_folds, shuffle = True)

    # Iterate over folds
    cv_rsquared = 0
    
    for train, test in kfold:
        x_train_cv = x.iloc[train].values
        y_train_cv = y.iloc[train].values
        x_test_cv = x.iloc[test].values
        y_test_cv = y.iloc[test].values
        
        # Fit linear regression model and evaluate
        knn_model = KNN(n_neighbors = n_neighbors)
        knn_model.fit(x_train_cv, y_train_cv)
        rsquared = knn_model.score(x_test_cv, y_test_cv)
    
        # Cummulative accuracy across folds
        cv_rsquared += rsquared

    # Return average accuracy across folds
    return cv_rsquared * 1.0 / num_folds

In [14]:
for i in range(2, 20): 
    rsquared = k_fold_knn (x_train, y_train, i, 3)
    print "R-squared for k=" + str(i) + " neighbors: " + str(rsquared)

ValueError: could not convert string to float: #NAME?

In [15]:
final_knn_model = KNN(n_neighbors = 15)
final_knn_model.fit(x_train.values, y_train.values)

ValueError: could not convert string to float: #NAME?

In [374]:
test_data = pd.read_csv("test_imputed.csv")

In [375]:
x_test = test_data.drop(['zestimate_amount'],1)
y_test = test_data['zestimate_amount']

In [376]:
final_knn_model.score(x_test.values, y_test.values)

0.25173119973659253

In [9]:
# Parameters for tuning
n_trees = np.arange(10, 100, 20)  # Trees and depth are explored on an exponentially growing space,
depths = np.arange(2, 10)   # since it is assumed that trees and depth will add accuracy in a decaying fashion.
# To keep track of the best model
best_score = 0
# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'
for trees in n_trees:
    for depth in depths:
        
        # Cross validation for every experiment
        k_folds = cross_validation.KFold(x_train.shape[0], n_folds=5, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = x_train[train_indices]
            y_train_cv = y_train[train_indices]
            # Generate validation data
            x_validate = x_train[validation_indices]
            y_validate = y_train[validation_indices]
            
            # Fit random forest on training data
            model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=depth)
            model.fit(x_train_cv, y_train_cv)
            # Score on validation data
            scores += [model.score(x_validate, y_validate)]
        
        # Record and report accuracy
        average_score = np.mean(scores)
        print "Trees:", trees, "Depth:", depth, "Score:", average_score
        
        # Update our record of the best parameters see so far
        if average_score > best_score:
            best_score = average_score
            best_trees = trees
            best_depth = depth
            
# Fit model on entire train set using chosen number of trees and depth
model = ensemble.RandomForestRegressor(n_estimators=best_trees, max_depth=best_depth)
model.fit(x_train, y_train)
print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x_test, y_test)


5-fold cross validation:


NameError: name 'x_train' is not defined